In [2]:
# Auto-install required packages
import importlib.util
import subprocess
import sys

def check_and_install_packages():
    required_packages = [
        'pandas',
        'requests',
        'office365-REST-Python-Client'
    ]
    
    print("Checking required packages...")
    for package in required_packages:
        if importlib.util.find_spec(package) is None:
            print(f"Installing {package}...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", package])
                print(f"Successfully installed {package}")
            except Exception as e:
                print(f"Error installing {package}: {str(e)}")
                print(f"Please install {package} manually: pip install {package}")
                sys.exit(1)
        else:
            print(f"✓ {package} already installed")
    print("All required packages are installed.")

# Run package check and installation before imports
check_and_install_packages()

# Now import the required packages
import pandas as pd
import requests
from io import StringIO, BytesIO
import numpy as np
import os
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import datetime
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Credentials ---
# Survey CTO credentials
username = "camille@reliefapplications.org"
password = "HELENKELLER2025!"

# SharePoint credentials
sharepoint_username = "rAssaf@HKI.org"
sharepoint_password = "R@ch^2627"

# --- SharePoint Configuration ---
sharepoint_site_url = "https://hkw.sharepoint.com/teams/PBI_MER_Data"
sharepoint_folder_path = "Shared Documents/python output"  # Fixed spelling

# --- Base URL ---
base_url = "https://keller2024.surveycto.com/api/v1/forms/data/csv"

# --- Fixed filename for Power BI connection ---
OUTPUT_FILENAME = "HKI_SurveyCTO_Data.csv"

# --- Country Form Configurations ---
country_configs = {
    "Burkina Faso": [
        {"form": "Monitorage_Menage_2024_R2", "round": "R2", "year": 2024},
        {"form": "Monitorage_Menage_2024_R2_v2", "round": "R2_v2", "year": 2024}
    ],
    "Cameroun": [
        {"form": "monitorage_menage_sasnim1_2024", "round": "SASNIM1", "year": 2024}
    ],
    "Côte d'Ivoire": [
        {"form": "ci2024_im_menage_r1_hki", "round": "R1_HKI", "year": 2024}
    ],
    "Guinée": [
        {"form": "gn2024_im_household_SVA_round1", "round": "SVA_round1", "year": 2024}
    ],
    "Mali": [
        {"form": "ml2024_im_household_sva_sabine_version3_round1", "round": "SVA_round1_v3", "year": 2024},
        {"form": "ml2024_im_household_sva_sabine_round1", "round": "SVA_round1", "year": 2024},
        {"form": "ml2024_im_household_sva_sabine_version2_round1", "round": "SVA_round1_v2", "year": 2024}
    ],
    "Niger": [
        {"form": "ne2024_im_household_round1", "round": "Round1", "year": 2024}
    ],
    "Nigeria": [
        {"form": "ng2024_im_menage_r2_hki", "round": "R2_HKI", "year": 2024}
    ],
    "RDC": [
        {"form": "cd2024_im_menage", "round": "R1", "year": 2024}
    ]
}

# --- EMBEDDED MAPPING DATA (cleaned to remove duplicates) ---
MAPPING_DATA = {
    "consent": {"Burkina Faso": "consentement", "Cameroun": "consentement", "Côte d'Ivoire": "consentement", "Guinée": "consentement", "Mali": "consentement", "Niger": "consentement", "Nigeria": "consentement", "RDC": "consentement"},
    "position": {"Burkina Faso": "acceptation-position", "Cameroun": "acceptation-position", "Côte d'Ivoire": "acceptation-position", "Guinée": "position", "Mali": "acceptation-position", "Niger": "acceptation-position", "Nigeria": "acceptation-position", "RDC": "acceptation-position"},
    "hh_number": {"Burkina Faso": "acceptation-q11", "Cameroun": "acceptation-q11", "Côte d'Ivoire": "acceptation-q11", "Guinée": "q11", "Mali": "acceptation-q11", "Niger": "acceptation-q11", "Nigeria": "acceptation-q11", "RDC": "acceptation-q11"},
    "visited": {"Burkina Faso": "acceptation-q12", "Cameroun": "acceptation-q12", "Côte d'Ivoire": "acceptation-q12", "Guinée": "q12", "Mali": "acceptation-q12", "Niger": "acceptation-q12", "Nigeria": "acceptation-q12", "RDC": "acceptation-q12"},
    
    # Q13 series - Burkina Faso doesn't have q13 columns, Cameroun has q13 without visite- prefix
    "marked": {"Cameroun": "acceptation-q13", "Côte d'Ivoire": "acceptation-q13", "Guinée": "q13", "Niger": "acceptation-visite-q13", "Nigeria": "acceptation-q13", "RDC": "acceptation-q13"},
    "marked_correct": {"Cameroun": "acceptation-q13a", "Côte d'Ivoire": "acceptation-q13a", "Guinée": "q13a", "Niger": "acceptation-visite-q13a", "RDC": "acceptation-q13a"},
    "social_mobilizer": {"Cameroun": "acceptation-q13b", "Côte d'Ivoire": "acceptation-q13b", "Guinée": "q13b", "Mali": "acceptation-q13b", "Niger": "acceptation-visite-q13b", "RDC": "acceptation-q13b"},
    "birth_certificate": {"Niger": "acceptation-visite-q13c"},
    "civil_registry": {"Cameroun": "acceptation-q13d", "Niger": "acceptation-visite-q13d", "RDC": "acceptation-q13d"},
    
    # Q14-Q20 series
    # NOTE: Cameroun has different numbering - missing q14, q16, q17
    "tot_6_11": {"Burkina Faso": "acceptation-q14", "Côte d'Ivoire": "acceptation-q14", "Guinée": "q14", "Mali": "acceptation-q14", "Niger": "acceptation-visite-q14", "Nigeria": "acceptation-q14", "RDC": "acceptation-q14"},
    "tot_12_59": {"Burkina Faso": "acceptation-q15", "Cameroun": "acceptation-q15", "Côte d'Ivoire": "acceptation-q15", "Guinée": "q15", "Mali": "acceptation-q15", "Niger": "acceptation-visite-q15", "Nigeria": "acceptation-q15", "RDC": "acceptation-q15"},
    "tot_6_59": {"Burkina Faso": "acceptation-q16", "Côte d'Ivoire": "acceptation-q16", "Guinée": "q16", "Mali": "acceptation-q16", "Niger": "acceptation-visite-q16", "Nigeria": "acceptation-q16", "RDC": "acceptation-q16"},
    "present_6_11": {"Burkina Faso": "acceptation-q17", "Côte d'Ivoire": "acceptation-q17", "Guinée": "q17", "Mali": "acceptation-q17", "Niger": "acceptation-visite-q17", "Nigeria": "acceptation-q17", "RDC": "acceptation-q17"},
    "present_12_59": {"Burkina Faso": "acceptation-q18", "Cameroun": "acceptation-q18", "Côte d'Ivoire": "acceptation-q18", "Guinée": "q18", "Mali": "acceptation-q18", "Niger": "acceptation-visite-q18", "Nigeria": "acceptation-q18", "RDC": "acceptation-q18"},
    "present_6_59": {"Burkina Faso": "acceptation-q19", "Cameroun": "acceptation-q19", "Côte d'Ivoire": "acceptation-q19", "Guinée": "q19", "Mali": "acceptation-q19", "Niger": "acceptation-visite-q19", "Nigeria": "acceptation-q19", "RDC": "acceptation-q19"},
    "interventions": {"Burkina Faso": "acceptation-q20", "Cameroun": "acceptation-q20", "Côte d'Ivoire": "acceptation-q20", "Guinée": "q20", "Mali": "acceptation-q20", "Niger": "acceptation-visite-q20", "Nigeria": "acceptation-q20", "RDC": "acceptation-q20"},
    
    # Additional Q20 variants
    "q20a_capital": {"Nigeria": "acceptation-Q20a"},  # Nigeria has capital Q
    "q20x": {"Côte d'Ivoire": "acceptation-q20x", "Guinée": "q20x"},
    
    # Q21 series - Vitamin A supplementation
    # NOTE: Cameroun missing q21a
    "supp_6_11": {"Burkina Faso": "acceptation-q21a", "Côte d'Ivoire": "acceptation-q21a", "Guinée": "q21a", "Mali": "acceptation-q21a", "Niger": "acceptation-visite-vitaA-q21a", "Nigeria": "acceptation-q21a", "RDC": "acceptation-q21a"},
    "supp_12_59": {"Burkina Faso": "acceptation-q21b", "Cameroun": "acceptation-q21b", "Côte d'Ivoire": "acceptation-q21b", "Guinée": "q21b", "Mali": "acceptation-q21b", "Niger": "acceptation-visite-vitaA-q21b", "Nigeria": "acceptation-q21b", "RDC": "acceptation-q21b"},
    "supp_6_59": {"Burkina Faso": "acceptation-q21c", "Cameroun": "acceptation-q21c", "Côte d'Ivoire": "acceptation-q21c", "Guinée": "q21c", "Mali": "acceptation-q21c", "Niger": "acceptation-visite-vitaA-q21c", "Nigeria": "acceptation-q21c", "RDC": "acceptation-q21c"},
    
    # Additional Q21 variants for RDC and Cameroun
    "q21a1": {"RDC": "acceptation-q21a1"},
    "q21b1": {"Cameroun": "acceptation-q21b1", "RDC": "acceptation-q21b1"},
    "q21d": {"RDC": "acceptation-q21d"},
    "q21e": {"RDC": "acceptation-q21e"},
    "q21f": {"Cameroun": "acceptation-q21f", "RDC": "acceptation-q21f"},
    
    # Q22 series - Not supplemented
    # NOTE: Cameroun missing q22a
    "not_supp_6_11": {"Burkina Faso": "acceptation-q22a", "Côte d'Ivoire": "acceptation-q22a", "Guinée": "q22a", "Mali": "q22a", "Niger": "acceptation-visite-vitaA-q22a", "Nigeria": "acceptation-q22a", "RDC": "acceptation-q22a"},
    "not_supp_12_59": {"Burkina Faso": "acceptation-q22b", "Cameroun": "acceptation-q22b", "Côte d'Ivoire": "acceptation-q22b", "Guinée": "q22b", "Mali": "q22b", "Niger": "acceptation-visite-vitaA-q22b", "Nigeria": "acceptation-q22b", "RDC": "acceptation-q22b"},
    "not_supp_6_59": {"Burkina Faso": "acceptation-total_nv", "Cameroun": "acceptation-total_nv", "Côte d'Ivoire": "acceptation-total_nv", "Guinée": "total_nv", "Mali": "total_nv", "Niger": "acceptation-visite-vitaA-total_nv", "Nigeria": "acceptation-total_nv", "RDC": "acceptation-total_nv"},
    
    # Additional Q22c for Mali
    "q22c": {"Mali": "q22c"},
    
    # Q23 series - Never supplemented
    # NOTE: Cameroun missing q23a, uses paravant- prefix for q23b/c
    "never_supp_6_11": {"Burkina Faso": "acceptation-q23a", "Côte d'Ivoire": "acceptation-q23a", "Guinée": "paravant-q23a", "Mali": "q23a", "Niger": "acceptation-visite-vitaA-q23a", "Nigeria": "acceptation-nosup-q23a", "RDC": "acceptation-paravant-q23a"},
    "never_supp_12_59": {"Burkina Faso": "acceptation-q23b", "Cameroun": "acceptation-paravant-q23b", "Côte d'Ivoire": "acceptation-q23b", "Guinée": "paravant-q23b", "Mali": "q23b", "Niger": "acceptation-visite-vitaA-q23b", "Nigeria": "acceptation-nosup-q23b", "RDC": "acceptation-paravant-q23b"},
    "never_supp_6_59": {"Burkina Faso": "acceptation-q23c", "Cameroun": "acceptation-paravant-q23c", "Côte d'Ivoire": "acceptation-q23c", "Guinée": "paravant-q23c", "Mali": "q23c", "Niger": "acceptation-visite-vitaA-q23c", "Nigeria": "acceptation-nosup-q23c", "RDC": "acceptation-paravant-q23c"},
    
    # Q24 series - Reasons for no supplementation
    "supp_absent": {"Burkina Faso": "acceptation-nosup-q24a", "Cameroun": "acceptation-nosup-q24a", "Côte d'Ivoire": "acceptation-nosup-q24a", "Guinée": "nosup-q24a", "Mali": "nosup-q24a", "Niger": "acceptation-visite-vitaA-nosup-q24a", "Nigeria": "acceptation-nosup-q24_reason-q24a", "RDC": "acceptation-nosup-q24a"},
    "supp_refusal": {"Burkina Faso": "acceptation-nosup-q24b", "Cameroun": "acceptation-nosup-q24b", "Côte d'Ivoire": "acceptation-nosup-q24b", "Guinée": "nosup-q24b", "Mali": "nosup-q24b", "Niger": "acceptation-visite-vitaA-nosup-q24b", "Nigeria": "acceptation-nosup-q24_reason-q24b", "RDC": "acceptation-nosup-q24b"},
    "supp_no_visit": {"Burkina Faso": "acceptation-nosup-q24c", "Cameroun": "acceptation-nosup-q24c", "Côte d'Ivoire": "acceptation-nosup-q24c", "Guinée": "nosup-q24c", "Mali": "nosup-q24c", "Niger": "acceptation-visite-vitaA-nosup-q24c", "Nigeria": "acceptation-nosup-q24_reason-q24c", "RDC": "acceptation-nosup-q24c"},
    "supp_omission": {"Burkina Faso": "acceptation-nosup-q24d", "Cameroun": "acceptation-nosup-q24d", "Côte d'Ivoire": "acceptation-nosup-q24d", "Guinée": "nosup-q24d", "Mali": "nosup-q24d", "Niger": "acceptation-visite-vitaA-nosup-q24d", "Nigeria": "acceptation-nosup-q24_reason-q24d", "RDC": "acceptation-nosup-q24d"},
    "supp_shortage": {"Burkina Faso": "acceptation-nosup-q24e", "Cameroun": "acceptation-nosup-q24e", "Côte d'Ivoire": "acceptation-nosup-q24e", "Guinée": "nosup-q24e", "Mali": "nosup-q24e", "Niger": "acceptation-visite-vitaA-nosup-q24e", "Nigeria": "acceptation-nosup-q24_reason-q24e", "RDC": "acceptation-nosup-q24e"},
    "supp_no_info": {"Burkina Faso": "acceptation-nosup-q24f", "Cameroun": "acceptation-nosup-q24f", "Côte d'Ivoire": "acceptation-nosup-q24f", "Guinée": "nosup-q24f", "Mali": "nosup-q24f", "Niger": "acceptation-visite-vitaA-nosup-q24f", "Nigeria": "acceptation-nosup-q24_reason-q24f", "RDC": "acceptation-nosup-q24f"},
    "no_supp_other": {"Burkina Faso": "acceptation-nosup-q24g", "Cameroun": "acceptation-nosup-q24g", "Côte d'Ivoire": "acceptation-nosup-q24x", "Mali": "nosup-q24g", "Niger": "acceptation-visite-vitaA-nosup-q24g", "Nigeria": "acceptation-nosup-q24_reason-q24g", "RDC": "acceptation-nosup-q24g"},
    "no_supp_other_text": {"Burkina Faso": "acceptation-q24g_autre", "Cameroun": "acceptation-nosup-q24g_autre", "Mali": "nosup-q24g_autre", "Niger": "acceptation-visite-vitaA-nosup-q24g_autre", "Nigeria": "acceptation-nosup-q24_reason-q24g_autre", "RDC": "acceptation-nosup-q24g_autre"},
    
    # Q24b series - Refusal reasons
    "supp_refusal_religious": {"Burkina Faso": "acceptation-refus-q24b1", "Cameroun": "acceptation-refus-q24b1", "Côte d'Ivoire": "acceptation-refus-q24b1", "Guinée": "refus-q24b1", "Mali": "refus-q24b1", "Niger": "acceptation-visite-vitaA-refus-q24b1", "Nigeria": "acceptation-refus-q24b1", "RDC": "acceptation-refus-q24b1"},
    "supp_refusal_sick_child": {"Burkina Faso": "acceptation-refus-q24b2", "Cameroun": "acceptation-refus-q24b2", "Côte d'Ivoire": "acceptation-refus-q24b2", "Guinée": "refus-q24b2", "Mali": "refus-q24b2", "Niger": "acceptation-visite-vitaA-refus-q24b2", "Nigeria": "acceptation-refus-q24b2", "RDC": "acceptation-refus-q24b2"},
    "supp_refusal_do_not_decide": {"Burkina Faso": "acceptation-refus-q24b3", "Cameroun": "acceptation-refus-q24b3", "Côte d'Ivoire": "acceptation-refus-q24b3", "Guinée": "refus-q24b3", "Mali": "refus-q24b3", "Niger": "acceptation-visite-vitaA-refus-q24b3", "Nigeria": "acceptation-refus-q24b3", "RDC": "acceptation-refus-q24b3"},
    "supp_refusal_dangerous": {"Burkina Faso": "acceptation-refus-q24b4", "Cameroun": "acceptation-refus-q24b4", "Côte d'Ivoire": "acceptation-refus-q24b4", "Guinée": "refus-q24b4", "Mali": "refus-q24b4", "Niger": "acceptation-visite-vitaA-refus-q24b4", "Nigeria": "acceptation-refus-q24b4", "RDC": "acceptation-refus-q24b4"},
    "supp_refusal_other": {"Burkina Faso": "acceptation-refus-q24b5", "Cameroun": "acceptation-refus-q24b5", "Côte d'Ivoire": "acceptation-refus-q24b5", "Guinée": "refus-q24b5", "Mali": "refus-q24b5", "Niger": "acceptation-visite-vitaA-refus-q24b5", "Nigeria": "acceptation-refus-q24b5", "RDC": "acceptation-refus-q24b5"},
    "supp_refusal_other_text": {"Burkina Faso": "acceptation-q24b6-autre", "Cameroun": "acceptation-refus-q24b6_autre", "Côte d'Ivoire": "acceptation-refus-q24b6-autre", "Guinée": "refus-q24b6-autre", "Mali": "refus-q24b6-autre", "Niger": "acceptation-visite-vitaA-refus-q24b6-autre", "Nigeria": "acceptation-refus-q24b6-autre", "RDC": "acceptation-refus-q24b6_autre"},
    
    # Additional Q24b6 for Cameroun and RDC
    "supp_refusal_q24b6": {"Cameroun": "acceptation-refus-q24b6", "RDC": "acceptation-refus-q24b6"},
    
    # Q25-Q26 series
    "q25": {"Burkina Faso": "acceptation-q25", "Cameroun": "acceptation-q25", "Côte d'Ivoire": "acceptation-q25", "Guinée": "q25", "Mali": "q25", "Niger": "acceptation-visite-vitaA-q25", "Nigeria": "acceptation-q25", "RDC": "acceptation-q25"},
    "q26": {"Burkina Faso": "acceptation-q26", "Cameroun": "acceptation-q26", "Côte d'Ivoire": "acceptation-q26", "Guinée": "q26", "Mali": "q26", "Niger": "acceptation-visite-vitaA-q26", "Nigeria": "acceptation-q26", "RDC": "acceptation-q26"},
    "vit_a_reasons_other": {"Burkina Faso": "acceptation-q26_autre", "Cameroun": "acceptation-q26_autre", "Côte d'Ivoire": "acceptation-q26_autre", "Guinée": "q26_autre", "Mali": "q26_autre", "Niger": "acceptation-visite-vitaA-q26_autre", "Nigeria": "acceptation-q26_autre", "RDC": "acceptation-q26_autre"},
    
    # Deworming series
    "dw_12_59": {"Burkina Faso": "acceptation-dep_group-q27", "Côte d'Ivoire": "acceptation-dep_group-q27", "Mali": "dep_group-q27", "Niger": "acceptation-visite-dep_group-q27", "Nigeria": "acceptation-dep_group-q27", "RDC": "acceptation-dep_group-q27"},
    "not_dw_12_59": {"Burkina Faso": "acceptation-dep_group-q28", "Cameroun": "acceptation-q28", "Côte d'Ivoire": "acceptation-dep_group-q28", "Mali": "dep_group-q28", "Niger": "acceptation-visite-dep_group-q28", "Nigeria": "acceptation-dep_group-q28", "RDC": "acceptation-dep_group-q28"},
    "q28a": {"RDC": "acceptation-dep_group-q28a"},
    
    # Q31 series - Deworming reasons
    "dw_absent": {"Burkina Faso": "acceptation-dep_group-noalb-q31a", "Mali": "no_alb-q31a", "Niger": "acceptation-visite-dep_group-noalb-q31a", "Nigeria": "acceptation-dep_group-noalb-q31a", "RDC": "acceptation-dep_group-noalb-q31a"},
    "dw_refusal": {"Burkina Faso": "acceptation-dep_group-noalb-q31b", "Mali": "no_alb-q31b", "Niger": "acceptation-visite-dep_group-noalb-q31b", "Nigeria": "acceptation-dep_group-noalb-q31b", "RDC": "acceptation-dep_group-noalb-q31b"},
    "dw_no_visit": {"Burkina Faso": "acceptation-dep_group-noalb-q31c", "Mali": "no_alb-q31c", "Niger": "acceptation-visite-dep_group-noalb-q31c", "Nigeria": "acceptation-dep_group-noalb-q31c", "RDC": "acceptation-dep_group-noalb-q31c"},
    "dw_omission": {"Burkina Faso": "acceptation-dep_group-noalb-q31d", "Mali": "no_alb-q31d", "Niger": "acceptation-visite-dep_group-noalb-q31d", "Nigeria": "acceptation-dep_group-noalb-q31d", "RDC": "acceptation-dep_group-noalb-q31d"},
    "dw_shortage": {"Burkina Faso": "acceptation-dep_group-noalb-q31e", "Mali": "no_alb-q31e", "Niger": "acceptation-visite-dep_group-noalb-q31e", "Nigeria": "acceptation-dep_group-noalb-q31e", "RDC": "acceptation-dep_group-noalb-q31e"},
    "dw_no_info": {"Burkina Faso": "acceptation-dep_group-noalb-q31f", "Mali": "no_alb-q31f", "Niger": "acceptation-visite-dep_group-noalb-q31f", "Nigeria": "acceptation-dep_group-noalb-q31f", "RDC": "acceptation-dep_group-noalb-q31f"},
    "dw_other": {"Burkina Faso": "acceptation-dep_group-noalb-q31g", "Mali": "no_alb-q31g", "Niger": "acceptation-visite-dep_group-noalb-q31g", "Nigeria": "acceptation-dep_group-noalb-q31g", "RDC": "acceptation-dep_group-noalb-q31g"},
    "dw_other_text": {"Burkina Faso": "acceptation-dep_group-q31g_autre", "Mali": "no_alb-q31g_autre", "Niger": "acceptation-visite-dep_group-noalb-q31g_autre", "Nigeria": "acceptation-dep_group-noalb-q31g_autre", "RDC": "acceptation-dep_group-noalb-q31g_autre"},
    "dw_why": {"Burkina Faso": "acceptation-dep_group-q32", "Mali": "q32", "Niger": "acceptation-visite-dep_group-q32", "Nigeria": "acceptation-dep_group-q32", "RDC": "acceptation-dep_group-q32"},
    "dw_why_list": {"Burkina Faso": "acceptation-dep_group-q33", "Mali": "q33", "Niger": "acceptation-visite-dep_group-q33", "Nigeria": "acceptation-dep_group-q33", "RDC": "acceptation-dep_group-q33"},
    "dw_why_other": {"Burkina Faso": "acceptation-dep_group-q33_autre", "Mali": "q33_autre", "Niger": "acceptation-visite-dep_group-q33_autre", "Nigeria": "acceptation-dep_group-q33_autre", "RDC": "acceptation-dep_group-q33_autre"},
    
    # Information campaign
    "info_campaign": {"Burkina Faso": "acceptation-q34", "Cameroun": "acceptation-q34", "Côte d'Ivoire": "acceptation-q34", "Guinée": "q34", "Mali": "q34", "Niger": "acceptation-visite-q34", "Nigeria": "acceptation-q34", "RDC": "acceptation-q34"},
    "info_next_campaign": {"Burkina Faso": "acceptation-q34a", "Cameroun": "acceptation-q34a", "Guinée": "q34a", "Niger": "acceptation-visite-q34a", "RDC": "acceptation-q34a"},
    "info_how": {"Burkina Faso": "acceptation-q35", "Cameroun": "acceptation-q35", "Côte d'Ivoire": "acceptation-q35", "Guinée": "q35", "Mali": "q35", "Niger": "acceptation-visite-q35", "Nigeria": "acceptation-q35", "RDC": "acceptation-q35"},
    "info_how_other": {"Burkina Faso": "acceptation-q35_autre", "Cameroun": "acceptation-q35_autre", "Côte d'Ivoire": "acceptation-q35_autre", "Guinée": "q35_autre", "Mali": "q35_autre", "Niger": "acceptation-visite-q35_autre", "Nigeria": "acceptation-q35_autre", "RDC": "acceptation-q35_autre"},
    "q35a": {"Mali": "q35a"},  # Additional for Mali
    "info_prefer": {"Burkina Faso": "acceptation-q36", "Cameroun": "acceptation-q36", "Côte d'Ivoire": "acceptation-q36", "Guinée": "q36", "Mali": "q36", "Niger": "acceptation-visite-q36", "Nigeria": "acceptation-q36", "RDC": "acceptation-q36"},
    "info_prefer_other": {"Burkina Faso": "acceptation-q36_autre", "Cameroun": "acceptation-q36_autre", "Côte d'Ivoire": "acceptation-q36_autre", "Guinée": "q36_autre", "Mali": "q36_autre", "Niger": "acceptation-visite-q36_autre", "RDC": "acceptation-q36_autre"},
    
    "q37": {"Burkina Faso": "acceptation-q37", "Cameroun": "acceptation-q37", "Côte d'Ivoire": "acceptation-q37", "Guinée": "q37", "Mali": "q37", "RDC": "acceptation-q37"},
    "q37c": {"Burkina Faso": "acceptation-q37c"},
    "q37c_pourquoi": {"Burkina Faso": "acceptation-q37c_pourquoi"},
    
    # Malnutrition screening
    "depistage": {"Burkina Faso": "acceptation-depistage", "Mali": "depistage", "Niger": "acceptation-visite-depistage", "Nigeria": "acceptation-depistage"},
    "malnutrition_target": {"Burkina Faso": "acceptation-depistage_group-q50", "Mali": "q50", "Niger": "acceptation-visite-depistage_group-q50", "Nigeria": "acceptation-depistage_group-q50"},
    "no_malnutrition": {"Burkina Faso": "acceptation-depistage_group-q51", "Mali": "q51", "Niger": "acceptation-visite-depistage_group-q51", "Nigeria": "acceptation-depistage_group-q51"},
    "52a": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52a", "Mali": "q52_group-q52a", "Niger": "acceptation-visite-depistage_group-q52_group-q52a", "Nigeria": "acceptation-depistage_group-q52_group-q52a"},
    "52b": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52b", "Mali": "q52_group-q52b", "Niger": "acceptation-visite-depistage_group-q52_group-q52b", "Nigeria": "acceptation-depistage_group-q52_group-q52b"},
    "52c": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52c", "Mali": "q52_group-q52c", "Niger": "acceptation-visite-depistage_group-q52_group-q52c", "Nigeria": "acceptation-depistage_group-q52_group-q52c"},
    "52d": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52d", "Mali": "q52_group-q52d", "Niger": "acceptation-visite-depistage_group-q52_group-q52d", "Nigeria": "acceptation-depistage_group-q52_group-q52d"},
    "52e": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52e", "Mali": "q52_group-q52e", "Niger": "acceptation-visite-depistage_group-q52_group-q52e", "Nigeria": "acceptation-depistage_group-q52_group-q52e"},
    "52f": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52f", "Mali": "q52_group-q52f", "Niger": "acceptation-visite-depistage_group-q52_group-q52f", "Nigeria": "acceptation-depistage_group-q52_group-q52f"},
    "52g": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52g", "Mali": "q52_group-q52g", "Niger": "acceptation-visite-depistage_group-q52_group-q52g", "Nigeria": "acceptation-depistage_group-q52_group-q52g"},
    "52h": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52h", "Mali": "q52_group-q52h", "Niger": "acceptation-visite-depistage_group-q52_group-q52h", "Nigeria": "acceptation-depistage_group-q52_group-q52h"},
    "52i": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52i", "Mali": "q52_group-q52i", "Niger": "acceptation-visite-depistage_group-q52_group-q52i", "Nigeria": "acceptation-depistage_group-q52_group-q52i"},
    "malnutrition_other": {"Burkina Faso": "acceptation-depistage_group-q52_group-q52i_autre", "Mali": "q52_group-q52i_autre", "Niger": "acceptation-visite-depistage_group-q52_group-q52i_autre", "Nigeria": "acceptation-depistage_group-q52_group-q52i_autre"},
    "malnutrition_why": {"Burkina Faso": "acceptation-depistage_group-q53", "Mali": "q53", "Niger": "acceptation-visite-depistage_group-q53", "Nigeria": "acceptation-depistage_group-q53"},
    "malnutrition_why_list": {"Burkina Faso": "acceptation-depistage_group-q54", "Mali": "q54", "Niger": "acceptation-visite-depistage_group-q54", "Nigeria": "acceptation-depistage_group-q54"},
    "malnutrition_why_other": {"Burkina Faso": "acceptation-depistage_group-q54_autre", "Mali": "q54_autre", "Niger": "acceptation-visite-depistage_group-q54_autre", "Nigeria": "acceptation-depistage_group-q54_autre"},
    
    # Q60+ series for Cameroun and RDC
    "q62": {"Cameroun": "acceptation-q62", "RDC": "acceptation-q62"},
    "q63": {"Cameroun": "acceptation-q63", "RDC": "acceptation-q63"},
    "q64": {"Cameroun": "acceptation-q64", "RDC": "acceptation-q64"},
    "q65": {"Cameroun": "acceptation-q65", "RDC": "acceptation-q65"},
    "q65_other": {"Cameroun": "acceptation-q65_autre", "RDC": "acceptation-q65_autre"},
    "q66": {"Cameroun": "acceptation-q66", "RDC": "acceptation-q66"},
    "q66_other": {"Cameroun": "acceptation-q66_autre", "RDC": "acceptation-q66_autre"},
    "q67": {"Cameroun": "acceptation-q67", "RDC": "acceptation-q67"},
    "q67_other": {"Cameroun": "acceptation-q67_autre", "RDC": "acceptation-q67_autre"},
    
    # Administrative/facility data
    "health_facility": {"Burkina Faso": "acceptation-q10c", "Cameroun": "acceptation-q10c", "Côte d'Ivoire": "acceptation-q10c", "Guinée": "q10c", "Mali": "q10c", "Niger": "acceptation-q10c", "Nigeria": "acceptation-q10c", "RDC": "acceptation-q10c"},
    "tot_children_supplemented": {"Burkina Faso": "acceptation-q10b", "Cameroun": "acceptation-q10b", "Côte d'Ivoire": "acceptation-q10b", "Mali": "q10b", "Niger": "acceptation-q10b", "Nigeria": "acceptation-q10b", "RDC": "acceptation-q10b"},
    "q10c1": {"Burkina Faso": "acceptation-q10c1", "Cameroun": "acceptation-q10c1", "RDC": "acceptation-q10c1"},
    "q10c2": {"Cameroun": "acceptation-q10c2", "RDC": "acceptation-q10c2"},
    "acceptation-q10c_pourquoi": {"Burkina Faso": "acceptation-q10c_pourquoi"},
    
    # Summary counts
    "no_supp_total_target": {"Burkina Faso": "acceptation-nb_vita", "Cameroun": "acceptation-nb_vita", "Côte d'Ivoire": "acceptation-nb_vita", "Guinée": "nb_vita", "Mali": "nb_vita", "Niger": "acceptation-nb_vita", "Nigeria": "acceptation-nb_vita", "RDC": "acceptation-nb_vita"},
    "no_dw_total_target": {"Burkina Faso": "acceptation-nb_alb", "Mali": "nb_alb", "Niger": "acceptation-nb_alb", "Nigeria": "acceptation-nb_alb"},
    "hh_problem": {"Burkina Faso": "acceptation-problem", "Cameroun": "acceptation-problem", "Côte d'Ivoire": "acceptation-problem", "Guinée": "problem", "Mali": "problem", "Niger": "acceptation-problem", "Nigeria": "acceptation-problem", "RDC": "acceptation-problem"},
    "hh_visited": {"Burkina Faso": "acceptation-men_visit", "Cameroun": "acceptation-men_visit", "Côte d'Ivoire": "acceptation-men_visit", "Guinée": "men_visit", "Mali": "men_visit", "Niger": "acceptation-men_visit", "Nigeria": "acceptation-men_visit", "RDC": "acceptation-men_visit"},
    
    # Geographic coordinates
    "latitude": {"Burkina Faso": "acceptation-coordonne-Latitude", "Cameroun": "acceptation-coordonne-Latitude", "Côte d'Ivoire": "acceptation-coordonne-Latitude", "Guinée": "coordonne-Latitude", "Mali": "coordonne-Latitude", "Niger": "acceptation-coordonne-Latitude", "Nigeria": "acceptation-coordonne-Latitude", "RDC": "acceptation-coordonne-Latitude"},
    "longitude": {"Burkina Faso": "acceptation-coordonne-Longitude", "Cameroun": "acceptation-coordonne-Longitude", "Côte d'Ivoire": "acceptation-coordonne-Longitude", "Guinée": "coordonne-Longitude", "Mali": "coordonne-Longitude", "Niger": "acceptation-coordonne-Longitude", "Nigeria": "acceptation-coordonne-Longitude", "RDC": "acceptation-coordonne-Longitude"},
    "altitude": {"Burkina Faso": "acceptation-coordonne-Altitude", "Cameroun": "acceptation-coordonne-Altitude", "Côte d'Ivoire": "acceptation-coordonne-Altitude", "Guinée": "coordonne-Altitude", "Mali": "coordonne-Altitude", "Niger": "acceptation-coordonne-Altitude", "Nigeria": "acceptation-coordonne-Altitude", "RDC": "acceptation-coordonne-Altitude"},
    "accuracy": {"Burkina Faso": "acceptation-coordonne-Accuracy", "Cameroun": "acceptation-coordonne-Accuracy", "Côte d'Ivoire": "acceptation-coordonne-Accuracy", "Guinée": "coordonne-Accuracy", "Mali": "coordonne-Accuracy", "Niger": "acceptation-coordonne-Accuracy", "Nigeria": "acceptation-coordonne-Accuracy", "RDC": "acceptation-coordonne-Accuracy"},
    
    # Technical/system fields
    "parent_key": {"Burkina Faso": "PARENT_KEY", "Cameroun": "PARENT_KEY", "Côte d'Ivoire": "PARENT_KEY", "Guinée": "PARENT_KEY", "Mali": "PARENT_KEY", "Niger": "PARENT_KEY", "Nigeria": "PARENT_KEY", "RDC": "PARENT_KEY"},
    "key": {"Burkina Faso": "KEY", "Cameroun": "KEY", "Côte d'Ivoire": "KEY", "Guinée": "KEY", "Mali": "KEY", "Niger": "KEY", "Nigeria": "KEY", "RDC": "KEY"},
    "set_of_menage": {"Burkina Faso": "SET-OF-menage", "Cameroun": "SET-OF-menage", "Côte d'Ivoire": "SET-OF-menage", "Guinée": "SET-OF-menage", "Mali": "SET-OF-menage", "Niger": "SET-OF-menage", "Nigeria": "SET-OF-menage", "RDC": "SET-OF-menage"},
    "submission_date": {"Burkina Faso": "SubmissionDate", "Cameroun": "SubmissionDate", "Côte d'Ivoire": "SubmissionDate", "Guinée": "SubmissionDate", "Mali": "SubmissionDate", "Niger": "SubmissionDate", "Nigeria": "SubmissionDate", "RDC": "SubmissionDate"},
    "starttime": {"Burkina Faso": "starttime", "Cameroun": "starttime", "Côte d'Ivoire": "starttime", "Guinée": "starttime", "Mali": "starttime", "Niger": "starttime", "Nigeria": "starttime", "RDC": "starttime"},
    "endtime": {"Burkina Faso": "endtime", "Cameroun": "endtime", "Côte d'Ivoire": "endtime", "Guinée": "endtime", "Mali": "endtime", "Niger": "endtime", "Nigeria": "endtime", "RDC": "endtime"},
    "deviceid": {"Burkina Faso": "deviceid", "Cameroun": "deviceid", "Côte d'Ivoire": "deviceid", "Guinée": "deviceid", "Mali": "deviceid", "Niger": "deviceid", "Nigeria": "deviceid", "RDC": "deviceid"},
    "devicephonenum": {"Burkina Faso": "devicephonenum", "Cameroun": "devicephonenum", "Côte d'Ivoire": "devicephonenum", "Guinée": "devicephonenum", "Mali": "devicephonenum", "Niger": "devicephonenum", "Nigeria": "devicephonenum", "RDC": "devicephonenum"},
    "username": {"Burkina Faso": "username", "Cameroun": "username", "Côte d'Ivoire": "username", "Guinée": "username", "Mali": "username", "Niger": "username", "Nigeria": "username", "RDC": "username"},
    "device_info": {"Burkina Faso": "device_info", "Cameroun": "device_info", "Côte d'Ivoire": "device_info", "Guinée": "device_info", "Mali": "device_info", "Niger": "device_info", "Nigeria": "device_info", "RDC": "device_info"},
    "duration": {"Burkina Faso": "duration", "Cameroun": "duration", "Côte d'Ivoire": "duration", "Guinée": "duration", "Mali": "duration", "Niger": "duration", "Nigeria": "duration", "RDC": "duration"},
    "round": {"Cameroun": "round", "Côte d'Ivoire": "round", "Guinée": "round", "Niger": "round", "RDC": "round"},
    "caseid": {"Burkina Faso": "caseid", "Cameroun": "caseid", "Côte d'Ivoire": "caseid", "Guinée": "caseid", "Niger": "caseid", "Nigeria": "caseid", "RDC": "caseid"},
    "date_jour": {"Burkina Faso": "date_jour", "Cameroun": "date_jour", "Côte d'Ivoire": "date_jour", "Guinée": "date_jour", "Mali": "date_jour", "Niger": "date_jour", "Nigeria": "date_jour", "RDC": "date_jour"},
    
    # Alternative SET-OF patterns
    "set_of_monitorage_menage": {"Côte d'Ivoire": "SET-OF-collect-menage", "Guinée": "SET-OF-collect-menage", "Niger": "SET-OF-monitorage-menage"},
    "meta_instanceID": {"Burkina Faso": "meta-instanceID", "Cameroun": "meta-instanceID", "Côte d'Ivoire": "meta-instanceID", "Guinée": "meta-instanceID", "Mali": "meta-instanceID", "Niger": "meta-instanceID", "Nigeria": "meta-instanceID", "RDC": "meta-instanceID"},
    "formdef_version": {"Burkina Faso": "formdef_version", "Cameroun": "formdef_version", "Côte d'Ivoire": "formdef_version", "Guinée": "formdef_version", "Mali": "formdef_version", "Niger": "formdef_version", "Nigeria": "formdef_version", "RDC": "formdef_version"},
    "review_quality": {"Burkina Faso": "review_quality", "Cameroun": "review_quality", "Côte d'Ivoire": "review_quality", "Guinée": "review_quality", "Mali": "review_quality", "Niger": "review_quality", "Nigeria": "review_quality", "RDC": "review_quality"},
    
    # Administrative hierarchy
    "supervisor": {"Burkina Faso": "superviseur", "Cameroun": "supervisor", "Côte d'Ivoire": "collect-superviseur", "Guinée": "collect-superviseur", "Mali": "superviseur", "Niger": "monitorage-superviseur", "Nigeria": "superviseur_list", "RDC": "supervisor"},
    "moniteur": {"Burkina Faso": "moniteur", "Cameroun": "moniteur", "Côte d'Ivoire": "collect-moniteur", "Guinée": "collect-moniteur", "Mali": "moniteur", "Niger": "monitorage-moniteur", "Nigeria": "monitor_list"},
    "moniteur_name": {"Burkina Faso": "moniteur_name"},
    
    # Geographic hierarchy
    "region": {"Burkina Faso": "region", "Cameroun": "region", "Côte d'Ivoire": "collect-region", "Guinée": "collect-region", "Mali": "region", "Niger": "monitorage-region", "Nigeria": "state", "RDC": "region"},
    "district": {"Burkina Faso": "district", "Cameroun": "district", "Côte d'Ivoire": "collect-district", "Guinée": "collect-district", "Mali": "district", "Niger": "monitorage-district", "Nigeria": "lga", "RDC": "district"},
    "district_name": {"Burkina Faso": "district_name"},
    
    "aire_sante": {"Burkina Faso": "centre_sante", "Cameroun": "aire_sante", "Côte d'Ivoire": "collect-centre_sante", "Guinée": "collect-centre_sante", "Mali": "aire_sante", "Niger": "monitorage-centre_sante", "RDC": "centre_sante"},
    "sanitaire_name": {"Burkina Faso": "sanitaire_name"},
    "localite": {"Burkina Faso": "localite", "Cameroun": "localite", "Côte d'Ivoire": "collect-localite", "Guinée": "collect-localite", "Mali": "localite", "Niger": "monitorage-localite", "Nigeria": "localite", "RDC": "localite"},
    "tot_men_prob": {"Burkina Faso": "tot_men_prob", "Cameroun": "tot_men_prob", "Côte d'Ivoire": "collect-tot_men_prob", "Guinée": "collect-tot_men_prob", "Mali": "tot_men_prob", "Niger": "monitorage-tot_men_prob", "Nigeria": "tot_men_prob", "RDC": "tot_men_prob"},
    "tot_visit": {"Burkina Faso": "tot_visit", "Cameroun": "tot_visit", "Côte d'Ivoire": "collect-tot_visit", "Guinée": "collect-tot_visit", "Mali": "tot_visit", "Niger": "monitorage-tot_visit", "Nigeria": "tot_visit", "RDC": "tot_visit"},
    
    # Case sensitivity duplicates (handle both lowercase and uppercase)
    "Round": {"Burkina Faso": "Round"},
    "Year": {"Burkina Faso": "Year"},
    "Country": {"Burkina Faso": "Country"},
    "meta-instanceName": {"Burkina Faso": "meta-instanceName"},
    
    # Additional unique columns
    "team_lead": {"Cameroun": "team_lead", "RDC": "team_lead"},
    "enume_name": {"RDC": "enume_name"},
    "nom_label": {"Cameroun": "nom_label"},
    "country": {"Burkina Faso": "country", "Cameroun": "country", "RDC": "country"},
    "year": {"Burkina Faso": "year", "Cameroun": "year", "RDC": "year"},
    "health_facility_nigeria": {"Nigeria": "health_facility"},
    "dep": {"Burkina Faso": "acceptation-dep", "Mali": "dep", "Niger": "acceptation-visite-vitaA-dep", "Nigeria": "acceptation-dep"},
    
    # Additional columns for specific countries
    "date_moins2mois": {"Côte d'Ivoire": "date_moins2mois"},
    "set_of_concession": {"Burkina Faso": "SET-OF-concession"},
    "duration_interview_localite": {"Burkina Faso": "duration_interview_localite"},
    "start_time": {"Burkina Faso": "start_time"},
    "end_time": {"Burkina Faso": "end_time"},
    
    # Niger specific geographic coordinates in parent form
    "monitorage_coordgeo_latitude": {"Niger": "monitorage-coordgeo-Latitude"},
    "monitorage_coordgeo_longitude": {"Niger": "monitorage-coordgeo-Longitude"},
    "monitorage_coordgeo_altitude": {"Niger": "monitorage-coordgeo-Altitude"},
    "monitorage_coordgeo_accuracy": {"Niger": "monitorage-coordgeo-Accuracy"},
    "monitorage_obs": {"Niger": "monitorage-obs"},
    
    # Azithromycin series for Côte d'Ivoire
    "azithromycine": {"Côte d'Ivoire": "acceptation-azithro_sup-azithromycine"},
    "azithro_q41": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q41"},
    "azithro_q41b": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q41b"},
    "azithro_q42": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q42"},
    "azithro_q43": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q43"},
    "azithro_q44": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q44"},
    "azithro_q45a": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45a"},
    "azithro_q45b": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45b"},
    "azithro_q45c": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45c"},
    "azithro_q45d": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45d"},
    "azithro_q45e": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45e"},
    "azithro_q45f": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45f"},
    "azithro_q45g": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45g"},
    "azithro_q45h": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45h"},
    "azithro_q45x": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45x"},
    "azithro_q45i_autre": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q45_group-q45i_autre"},
    "azithro_q46a": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46a"},
    "azithro_q46b": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46b"},
    "azithro_q46c": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46c"},
    "azithro_q46d": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46d"},
    "azithro_q46e": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46e"},
    "azithro_q46f": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46f"},
    "azithro_q46f_autre": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q46_group-q46f_autre"},
    "azithro_q47": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q47"},
    "azithro_q47_autre": {"Côte d'Ivoire": "acceptation-azithro_sup-azithro_group-q47_autre"},
    
    # Additional new questions for Côte d'Ivoire
    "new_q1": {"Côte d'Ivoire": "acceptation-NewQ1"},
    "new_q2": {"Côte d'Ivoire": "acceptation-NewQ2"},
    "new_q3": {"Côte d'Ivoire": "acceptation-NewQ3"},
    "new_q4": {"Côte d'Ivoire": "acceptation-NewQ4"},
    "new_q5": {"Côte d'Ivoire": "acceptation-NewQ5"},
    
    # Additional Cameroun specific
    "q10c2autre": {"Cameroun": "acceptation-q10c2autre"}
}

def fetch_and_merge_form_data(form_config, country, auth_tuple):
    print(f"🔄 Processing {country} - {form_config['round']} - {form_config['form']}")
    
    # Fetch parent data
    parent_url = f"{base_url}/{form_config['form']}"
    print(f"  • Fetching parent data from: {parent_url}")
    r_parent = requests.get(parent_url, auth=auth_tuple)
    
    if r_parent.status_code != 200:
        print(f"  ⚠️ Failed to fetch parent data: HTTP {r_parent.status_code}")
        return None
        
    parent_df = pd.read_csv(StringIO(r_parent.text), low_memory=False)
    parent_df.columns = [col.strip() if isinstance(col, str) else col for col in parent_df.columns]
    print(f"  • Parent data: {parent_df.shape[0]} rows × {parent_df.shape[1]} columns")

    # Fetch child data
    child_url = f"{base_url}/{form_config['form']}/menage"
    print(f"  • Fetching child data from: {child_url}")
    r_child = requests.get(child_url, auth=auth_tuple)
    
    if r_child.status_code != 200:
        print(f"  ⚠️ Failed to fetch child data: HTTP {r_child.status_code}")
        print(f"  ⚠️ This form might not have child records. Using parent data only.")
        # If no child data, return parent data with metadata
        parent_df["Country"] = country
        parent_df["Round"] = form_config["round"]
        parent_df["Year"] = form_config["year"]
        return parent_df
        
    child_df = pd.read_csv(StringIO(r_child.text), low_memory=False)
    child_df.columns = [col.strip() if isinstance(col, str) else col for col in child_df.columns]
    print(f"  • Child data: {child_df.shape[0]} rows × {child_df.shape[1]} columns")

    # Check if child data has required columns
    if child_df.empty:
        print(f"  ⚠️ Child data is empty. Using parent data only.")
        parent_df["Country"] = country
        parent_df["Round"] = form_config["round"]
        parent_df["Year"] = form_config["year"]
        return parent_df
    
    # Check for PARENT_KEY column
    if "PARENT_KEY" not in child_df.columns:
        print(f"  ⚠️ PARENT_KEY column not found in child data.")
        print(f"  ⚠️ Available columns in child data: {list(child_df.columns)[:10]}...")
        
        # Try alternative column names
        possible_parent_keys = ["parent_key", "Parent_Key", "ParentKey", "PARENTKEY"]
        found_key = None
        for alt_key in possible_parent_keys:
            if alt_key in child_df.columns:
                found_key = alt_key
                print(f"  ✓ Found alternative parent key column: {alt_key}")
                break
        
        if not found_key:
            print(f"  ⚠️ No parent key column found. Using parent data only.")
            parent_df["Country"] = country
            parent_df["Round"] = form_config["round"]
            parent_df["Year"] = form_config["year"]
            return parent_df
        else:
            # Rename the found column to PARENT_KEY
            child_df = child_df.rename(columns={found_key: "PARENT_KEY"})

    # Check for KEY column in parent
    if "KEY" not in parent_df.columns:
        print(f"  ⚠️ KEY column not found in parent data.")
        print(f"  ⚠️ Available columns in parent data: {list(parent_df.columns)[:10]}...")
        # Try alternative column names
        possible_keys = ["key", "Key", "id", "ID", "_id"]
        found_key = None
        for alt_key in possible_keys:
            if alt_key in parent_df.columns:
                found_key = alt_key
                print(f"  ✓ Found alternative key column: {alt_key}")
                break
        
        if not found_key:
            print(f"  ⚠️ No key column found in parent data. Cannot merge.")
            parent_df["Country"] = country
            parent_df["Round"] = form_config["round"]
            parent_df["Year"] = form_config["year"]
            return parent_df
        else:
            parent_df = parent_df.rename(columns={found_key: "KEY"})

    # Drop existing Country, Round, Year columns if they exist
    for col in ['Country', 'Round', 'Year']:
        parent_df = parent_df.drop(columns=[col], errors='ignore')
        child_df = child_df.drop(columns=[col], errors='ignore')

    # Perform the merge
    try:
        merged_df = child_df.merge(parent_df, left_on="PARENT_KEY", right_on="KEY", how="left", suffixes=('', '_parent'))
        merged_df.drop(columns=['KEY_parent'], errors='ignore', inplace=True)
        merged_df["Country"] = country
        merged_df["Round"] = form_config["round"]
        merged_df["Year"] = form_config["year"]

        print(f"  ✓ Successfully merged: {merged_df.shape[0]} rows × {merged_df.shape[1]} columns")
        return merged_df
    except Exception as e:
        print(f"  ❌ Error during merge: {str(e)}")
        # Return parent data as fallback
        parent_df["Country"] = country
        parent_df["Round"] = form_config["round"]
        parent_df["Year"] = form_config["year"]
        return parent_df

def process_all_countries(country_configs, auth_tuple):
    country_dfs = {}
    for country, forms in country_configs.items():
        print(f"\n=== Processing {country} ===")
        country_merged_dfs = []
        for f in forms:
            result = fetch_and_merge_form_data(f, country, auth_tuple)
            if result is not None:
                country_merged_dfs.append(result)
        
        if country_merged_dfs:
            country_dfs[country] = pd.concat(country_merged_dfs, ignore_index=True)
            print(f"✅ Final {country} dataset: {country_dfs[country].shape[0]} rows × {country_dfs[country].shape[1]} columns")
        else:
            print(f"❌ No data collected for {country}")
    return country_dfs

def create_country_mapping_dict(mapping, country):
    """
    Creates a dictionary for a specific country where:
    - Key: source column name (from country column in mapping)
    - Value: target column name (from mapping keys)
    """
    country_mapping = {}
    for target_col, country_mappings in mapping.items():
        if country in country_mappings:
            source_col = country_mappings[country]
            if source_col and source_col != "":
                country_mapping[source_col] = target_col
    
    # Debug print for Nigeria
    if country == "Nigeria" and ('state' in country_mapping or 'lga' in country_mapping):
        print(f"  DEBUG: Nigeria mapping dict: {[(k, v) for k, v in country_mapping.items() if k in ['state', 'lga', 'region', 'district']]}")
    
    return country_mapping

def harmonize_country_df(df, mapping_dict, country_name):
    """
    Rename columns in a DataFrame according to the mapping, ensuring no duplicate column names.
    """
    df.columns = [col.strip() if isinstance(col, str) else col for col in df.columns]

    # Create valid mappings that exist in the DataFrame
    valid_mappings = {k.strip(): v for k, v in mapping_dict.items() if k.strip() in df.columns}

    # Check for duplicates in the mapped names
    mapped_values = list(valid_mappings.values())
    if len(mapped_values) != len(set(mapped_values)):
        raise ValueError(f"Duplicate target column names in mapping for {country_name}: {mapped_values}")

    print(f"  • Mapped {len(valid_mappings)} columns out of {len(df.columns)}")
    
    # Special logging for Nigeria
    if country_name == "Nigeria":
        nigeria_mappings = [(k, v) for k, v in valid_mappings.items() if k in ['state', 'lga'] or v in ['region', 'district']]
        if nigeria_mappings:
            print(f"  • Nigeria critical mappings: {nigeria_mappings}")

    # Rename columns and ensure uniqueness
    df = df.rename(columns=valid_mappings)
    df = df.loc[:, ~df.columns.duplicated()].copy()  # drop duplicate columns by name

    df['Country'] = country_name
    return df.reset_index(drop=True)

def harmonize_all_countries(country_dfs, mapping):
    harmonized_dfs = []
    for country, df in country_dfs.items():
        print(f"\nHarmonizing {country} dataset...")
        mapping_dict = create_country_mapping_dict(mapping, country)
        harmonized_dfs.append(harmonize_country_df(df, mapping_dict, country))
    combined_df = pd.concat(harmonized_dfs, ignore_index=True)
    print(f"\nFinal harmonized dataset: {combined_df.shape[0]} rows × {combined_df.shape[1]} columns")
    return combined_df

def apply_additional_transformations(df):
    # Replace applymap with more modern approach to avoid FutureWarning
    df = df.map(lambda x: str(x).replace(".0", "") if isinstance(x, str) else x)
    
    rename_map = {
        "tot_men_prob": "Number of hh with problems",
        "tot_visit": "Number of hh visited",
        "52a": "malnutrition_absent",
        "52b": "malnutrition_no_visit",
        "52c": "malnutrition_no_visit_again",
        "52d": "malnutrition_no_info",
        "52e": "malnutrition_ill",
        "52f": "malnutrition_refusal",
        "52g": "malnutrition_no_shakir",
        "52h": "malnutrition_dont_know",
        "52i": "malnutrition_young"
    }
    df = df.rename(columns=rename_map)
    if 'centre_sante' in df.columns and 'aire_sante' in df.columns:
        df["centre_sante"] = df.apply(
            lambda row: row["aire_sante"] if pd.isna(row["centre_sante"]) or str(row["centre_sante"]).strip() == "" else row["centre_sante"],
            axis=1
        )
    if 'submission_date' in df.columns:
        df["submission_date"] = pd.to_datetime(df["submission_date"], errors="coerce", format='mixed')
        df["Year_"] = df["submission_date"].dt.year
        df["Quarter_"] = df["submission_date"].dt.quarter

    # --- Column type definitions (FIXED for region/district) ---
    column_types = {
        'consent': 'Int64', 'position': 'Int64', 'hh_number': 'Int64', 
        'Number of hh with problems': 'Int64', 'Number of hh visited': 'Int64', 
        'malnutrition_absent': 'Int64', 'malnutrition_no_visit': 'Int64', 
        'malnutrition_no_visit_again': 'Int64', 'malnutrition_no_info': 'Int64', 
        'malnutrition_ill': 'Int64', 'malnutrition_refusal': 'Int64', 
        'malnutrition_no_shakir': 'Int64', 'malnutrition_dont_know': 'Int64', 
        'malnutrition_young': 'Int64', 'visited': 'Int64', 'tot_6_11': 'Int64', 
        'tot_12_59': 'Int64', 'tot_6_59': 'Int64', 'present_6_11': 'Int64', 
        'present_12_59': 'Int64', 'present_6_59': 'Int64', 'interventions': 'string', 
        'supp_6_11': 'Int64', 'supp_12_59': 'Int64', 'supp_6_59': 'Int64', 
        'not_supp_6_11': 'Int64', 'not_supp_12_59': 'Int64', 'not_supp_6_59': 'Int64', 
        'never_supp_6_11': 'Int64', 'never_supp_12_59': 'Int64', 'never_supp_6_59': 'Int64', 
        'supp_absent': 'Int64', 'supp_refusal': 'Int64', 'supp_no_visit': 'Int64', 
        'supp_omission': 'Int64', 'supp_shortage': 'Int64', 'supp_no_info': 'Int64', 
        'no_supp_other': 'Int64', 'no_supp_other_text': 'string', 
        'supp_refusal_religious': 'string', 'supp_refusal_sick_child': 'string', 
        'supp_refusal_do_not_decide': 'string', 'supp_refusal_dangerous': 'string', 
        'supp_refusal_other': 'string', 'supp_refusal_other_text': 'string', 
        'q25': 'string', 'q26': 'string', 'vit_a_reasons_other': 'string', 
        'acceptation-q10c': 'Int64', 'tot_children_supplemented': 'Int64', 
        'q10c1': 'Int64', 'acceptation-q10c_pourquoi': 'string', 
        'acceptation-dep': 'Int64', 'dw_12_59': 'Int64', 'not_dw_12_59': 'Int64', 
        'dw_absent': 'Int64', 'dw_refusal': 'Int64', 'dw_no_visit': 'Int64', 
        'dw_omission': 'Int64', 'dw_shortage': 'Int64', 'dw_no_info': 'Int64', 
        'dw_other': 'Int64', 'dw_other_text': 'string', 'dw_why': 'Int64', 
        'dw_why_list': 'string', 'dw_why_other': 'string', 'info_campaign': 'Int64', 
        'info_how': 'string', 'info_how_other': 'string', 'info_next_campaign': 'Int64', 
        'info_prefer': 'string', 'info_prefer_other': 'string', 'q37': 'Int64', 
        'q37c': 'Int64', 'q37c_pourquoi': 'string', 'depistage': 'Int64', 
        'malnutrition_target': 'Int64', 'no_malnutrition': 'Int64', 
        'malnutrition_other': 'string', 'malnutrition_why': 'Int64', 
        'malnutrition_why_list': 'string', 'malnutrition_why_other': 'string', 
        'no_supp_total_target': 'Int64', 'no_dw_total_target': 'Int64', 
        'hh_problem': 'Int64', 'hh_visited': 'Int64', 'latitude': 'float', 
        'longitude': 'float', 'altitude': 'float', 'accuracy': 'float', 
        'parent_key': 'string', 'key': 'string', 'set_of_menage': 'string', 
        'submission_date': 'datetime64[ns]', 'starttime': 'datetime64[ns]', 
        'endtime': 'datetime64[ns]', 'deviceid': 'string', 'devicephonenum': 'string', 
        'username': 'string', 'device_info': 'string', 'duration': 'Int64', 
        'date_jour': 'datetime64[ns]', 'supervisor': 'Int64', 'moniteur': 'Int64', 
        'moniteur_name': 'string', 
        'region': 'string',  # Changed from Int64 to string
        'district': 'string',  # Changed from Int64 to string
        'district_name': 'string', 'centre_sante': 'string', 'sanitaire_name': 'string', 
        'localite': 'string', 'set_of_monitorage_menage': 'string', 
        'meta_instanceID': 'string', 'meta-instanceName': 'string', 
        'formdef_version': 'Int64', 'review_quality': 'string', 
        'Country': 'string', 'Round': 'string', 'Year': 'Int64', 
        'marked': 'string', 'marked_correct': 'string', 
        'state': 'string', 'lga': 'string'  # Add these for Nigeria
    }

    # Apply column type conversions
    for col, dtype in column_types.items():
        if col in df.columns:
            try:
                if dtype == "Int64":
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
                elif dtype == "float":
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype("float")
                elif dtype == "datetime64[ns]":
                    df[col] = pd.to_datetime(df[col], errors="coerce")
                elif dtype == "string":
                    df[col] = df[col].astype("string")
            except Exception as e:
                print(f"Warning: Could not convert column '{col}' to {dtype}: {e}")
    return df

def check_missing_data(df):
    """Simple check for missing region and district data by country"""
    
    print("\n" + "="*60)
    print("CHECKING MISSING REGION AND DISTRICT INFORMATION")
    print("="*60)
    
    for country in df['Country'].unique():
        country_data = df[df['Country'] == country]
        total_records = len(country_data)
        
        print(f"\n{country}: {total_records:,} records")
        
        # Check region column if it exists
        if 'region' in df.columns:
            missing_region = country_data['region'].isna().sum()
            blank_region = (country_data['region'].astype(str).str.strip() == '').sum()
            total_missing_region = missing_region + blank_region
            if total_missing_region > 0:
                print(f"  Region missing: {total_missing_region:,} ({(total_missing_region/total_records)*100:.1f}%)")
            else:
                print(f"  Region: ✓ Complete")
        else:
            print(f"  Region: ⚠️ Column not found")
        
        # Check district column if it exists  
        if 'district' in df.columns:
            missing_district = country_data['district'].isna().sum()
            blank_district = (country_data['district'].astype(str).str.strip() == '').sum()
            total_missing_district = missing_district + blank_district
            if total_missing_district > 0:
                print(f"  District missing: {total_missing_district:,} ({(total_missing_district/total_records)*100:.1f}%)")
            else:
                print(f"  District: ✓ Complete")
        else:
            print(f"  District: ⚠️ Column not found")

def upload_to_sharepoint(file_content, filename, site_url, folder_path, username, password):
    """Upload file to SharePoint"""
    logging.info(f"Uploading/Overwriting {filename} to SharePoint")
    
    try:
        # Connect to SharePoint
        ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))
        
        # First, let's check what document libraries exist
        print("Checking available document libraries...")
        lists = ctx.web.lists
        ctx.load(lists)
        ctx.execute_query()
        
        doc_lib_names = []
        for lst in lists:
            if lst.properties.get('BaseTemplate') == 101:  # Document library template
                doc_lib_names.append(lst.properties.get('Title'))
        
        print(f"Found document libraries: {doc_lib_names}")
        
        # Try different possible document library names
        possible_lib_names = ["Documents", "Shared Documents", "Shared%20Documents", "Dokumente", "Documenten"]
        doc_lib = None
        
        for lib_name in possible_lib_names:
            if lib_name in doc_lib_names:
                try:
                    doc_lib = ctx.web.lists.get_by_title(lib_name)
                    ctx.load(doc_lib)
                    ctx.execute_query()
                    print(f"Using document library: {lib_name}")
                    break
                except:
                    continue
        
        if not doc_lib and doc_lib_names:
            # Use the first available document library
            doc_lib = ctx.web.lists.get_by_title(doc_lib_names[0])
            ctx.load(doc_lib)
            ctx.execute_query()
            print(f"Using first available document library: {doc_lib_names[0]}")
        
        if not doc_lib:
            raise Exception("No document library found")
        
        # Get root folder
        root_folder = doc_lib.root_folder
        ctx.load(root_folder)
        ctx.execute_query()
        
        # Try to create the subfolder
        try:
            # Check if folder exists
            target_folder = root_folder.folders.get_by_url("python output")
            ctx.load(target_folder)
            ctx.execute_query()
            print("Folder 'python output' already exists")
        except:
            # Create the folder
            print("Creating folder 'python output'...")
            target_folder = root_folder.folders.add("python output")
            ctx.execute_query()
            print("Folder created successfully")
        
        # Upload the file to the folder
        print(f"Uploading {filename}...")
        
        # Delete existing file if it exists
        try:
            existing_file = target_folder.files.get_by_url(filename)
            existing_file.delete_object()
            ctx.execute_query()
            print(f"Deleted existing file: {filename}")
        except:
            print(f"No existing file to delete")
        
        # Upload the new file
        uploaded_file = target_folder.upload_file(filename, file_content)
        ctx.execute_query()
        
        logging.info(f"Successfully uploaded {filename} to SharePoint")
        print(f"✅ File uploaded successfully to 'python output' folder")
        return True
        
    except Exception as e:
        logging.error(f"Error uploading to SharePoint: {str(e)}")
        print(f"SharePoint upload error: {str(e)}")
        
        # Try alternative approach - direct file upload to root
        try:
            print("\nTrying alternative upload method...")
            ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))
            
            # Try to upload directly to the site
            web = ctx.web
            ctx.load(web)
            ctx.execute_query()
            
            # Get the first document library
            lists = web.lists.filter("BaseTemplate eq 101")
            ctx.load(lists)
            ctx.execute_query()
            
            if len(lists) > 0:
                doc_lib = lists[0]
                # Upload directly to root folder
                uploaded_file = doc_lib.root_folder.upload_file(filename, file_content)
                ctx.execute_query()
                print(f"✅ File uploaded to root folder of document library")
                return True
        except Exception as e2:
            print(f"Alternative method also failed: {str(e2)}")
        
        return False

# --- Main execution ---
def main():
    try:
        # Process SurveyCTO data
        logging.info("Starting data collection from SurveyCTO")
        auth_tuple = (username, password)
        country_dfs = process_all_countries(country_configs, auth_tuple)
        
        if not country_dfs:
            print("❌ No data collected from any country")
            return
        
        # Use embedded mapping data directly
        print("\nUsing embedded mapping data (cleaned)")
        print("  DEBUG: Nigeria region mapping:", MAPPING_DATA.get("region", {}).get("Nigeria"))
        print("  DEBUG: Nigeria district mapping:", MAPPING_DATA.get("district", {}).get("Nigeria"))
        
        final_dataset = harmonize_all_countries(country_dfs, MAPPING_DATA)
        
        if final_dataset is not None:
            final_dataset = apply_additional_transformations(final_dataset)
            
            # Check for missing region and district data
            check_missing_data(final_dataset)
            
            # Save to buffer for SharePoint upload
            output_buffer = BytesIO()
            final_dataset.to_csv(output_buffer, index=False)
            output_buffer.seek(0)
            
            # Upload to SharePoint
            success = upload_to_sharepoint(
                output_buffer.getvalue(),
                OUTPUT_FILENAME,
                sharepoint_site_url,
                sharepoint_folder_path,
                sharepoint_username,
                sharepoint_password
            )
            
            if success:
                print(f"\n✅ Successfully uploaded {OUTPUT_FILENAME} to SharePoint")
            else:
                print(f"\n❌ Failed to upload to SharePoint")
                # Save locally as backup
                final_dataset.to_csv(OUTPUT_FILENAME, index=False)
                print(f"✅ Saved locally as {OUTPUT_FILENAME}")
                
        else:
            logging.error("❌ Failed to create final dataset")
            print("❌ Failed to create final dataset")
            
    except Exception as e:
        logging.error(f"❌ Error in execution: {str(e)}")
        print(f"❌ Error in execution: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Checking required packages...
✓ pandas already installed
✓ requests already installed
Installing office365-REST-Python-Client...


2025-06-20 11:03:52,432 - INFO - Starting data collection from SurveyCTO


All required packages are installed.

=== Processing Burkina Faso ===
🔄 Processing Burkina Faso - R2 - Monitorage_Menage_2024_R2
  • Fetching parent data from: https://keller2024.surveycto.com/api/v1/forms/data/csv/Monitorage_Menage_2024_R2
  • Parent data: 24 rows × 27 columns
  • Fetching child data from: https://keller2024.surveycto.com/api/v1/forms/data/csv/Monitorage_Menage_2024_R2/menage
  • Child data: 255 rows × 91 columns
  ✓ Successfully merged: 255 rows × 119 columns
🔄 Processing Burkina Faso - R2_v2 - Monitorage_Menage_2024_R2_v2
  • Fetching parent data from: https://keller2024.surveycto.com/api/v1/forms/data/csv/Monitorage_Menage_2024_R2_v2
  • Parent data: 57 rows × 33 columns
  • Fetching child data from: https://keller2024.surveycto.com/api/v1/forms/data/csv/Monitorage_Menage_2024_R2_v2/menage
  ⚠️ Failed to fetch child data: HTTP 400
  ⚠️ This form might not have child records. Using parent data only.
✅ Final Burkina Faso dataset: 312 rows × 127 columns

=== Processin

2025-06-20 11:04:10,782 - INFO - Uploading/Overwriting HKI_SurveyCTO_Data.csv to SharePoint


Checking available document libraries...
Found document libraries: ['Documents', 'Form Templates', 'Site Assets', 'Style Library']
Using document library: Documents
Folder 'python output' already exists
Uploading HKI_SurveyCTO_Data.csv...
Deleted existing file: HKI_SurveyCTO_Data.csv


2025-06-20 11:04:45,495 - INFO - Successfully uploaded HKI_SurveyCTO_Data.csv to SharePoint


✅ File uploaded successfully to 'python output' folder

✅ Successfully uploaded HKI_SurveyCTO_Data.csv to SharePoint
